In [5]:
import pandas as pd
import folium
from folium.plugins import HeatMap, Fullscreen, FloatImage
from sqlalchemy import create_engine
import utm
import rasterio
from rasterio.enums import Resampling
from streamlit_folium import folium_static
import matplotlib.pyplot as plt

# Detalles de la conexión
server = '192.168.200.31'
database = 'jmineops'
username = 'tinformacion'
password = 'Timeinlondon$'
conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(conn_str)

# Consulta SQL para seleccionar los primeros 100000 registros de la tabla
query = """
SELECT TOP 1000 [time]
      ,[Northing] AS j
      ,[Easting] AS i
      ,[Elevation]
      ,[speed] AS Speed
      ,[full_name]
      ,[Alarma]
  FROM [jmineops].[dbo].[T_sensors_speed_coord]
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
data = pd.read_sql(query, engine)

# Verificar cuántos registros se cargaron
print(f"Registros cargados: {data.shape[0]} filas y {data.shape[1]} columnas")

# Convertir las coordenadas UTM a latitud y longitud
def utm_to_latlon(row):
    lat, lon = utm.to_latlon(row['i'], row['j'], 13, 'N')  # Ajustar la zona y el hemisferio según sea necesario
    return pd.Series([lat, lon])

data[['latitude', 'longitude']] = data.apply(utm_to_latlon, axis=1)

# Mostrar las primeras filas después de la conversión de coordenadas
print("Después de convertir UTM a lat/lon:")
print(data.head())

# Filtrar datos para eliminar coordenadas inválidas
valid_lat_range = (18, 22)
valid_lon_range = (-107, -100)
data = data[(data['latitude'].between(*valid_lat_range)) & (data['longitude'].between(*valid_lon_range))]

# Convertir la columna 'time' a datetime
data['time'] = pd.to_datetime(data['time'])

# Eliminar filas con valores faltantes y velocidades mayores a 50 km/h
data = data.dropna(subset=['latitude', 'longitude', 'Speed', 'Alarma', 'Elevation'])
data = data[data['Speed'] <= 50]

# Verificar la cantidad de registros después de filtrar nulos y velocidad
print(f"Registros después de eliminar nulos y filtrar velocidad: {data.shape[0]} filas")

# Imprimir rango de fechas y lista de empleados para verificar
print("Fecha mínima en los datos:", data['time'].min())
print("Fecha máxima en los datos:", data['time'].max())
print("Empleados disponibles:", data['full_name'].unique())

# Ajustar los filtros de fecha y empleado
start_date = pd.to_datetime('2020-01-01')  # Dentro del rango de datos
end_date = pd.to_datetime('2020-01-20')    # Dentro del rango de datos
employee = 'Todos'  # Cambiar a un nombre específico si es necesario

# Filtrar datos según los filtros seleccionados
filtered_data = data[
    (data['time'] >= start_date) &
    (data['time'] <= end_date)
]

if employee != 'Todos':
    filtered_data = filtered_data[filtered_data['full_name'] == employee]

# Verificar si el DataFrame filtrado está vacío
if filtered_data.empty:
    print("El DataFrame filtrado está vacío. Verifica los filtros de fecha y empleado.")
else:
    # Imprimir estadísticas básicas del DataFrame filtrado
    print("Datos filtrados:")
    print(filtered_data.describe())
    print(filtered_data.head())

    # Crear una imagen de la barra de colores
    def create_colorbar():
        fig, ax = plt.subplots(figsize=(1, 8))
        cmap = plt.get_cmap('viridis')
        norm = plt.Normalize(vmin=0, vmax=50)
        fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=ax, orientation='vertical', label='Velocidad (km/h)')
        plt.savefig('colorbar.png', bbox_inches='tight', pad_inches=0.1)
        plt.close()

    create_colorbar()

    # Función para reducir el tamaño del GeoTIFF usando rasterio
    def reduce_geotiff_with_rasterio(input_path, output_path, scale_factor=0.5):
        """Reduce el tamaño del GeoTIFF usando rasterio."""
        with rasterio.open(input_path) as src:
            # Calcular nuevas dimensiones
            new_width = int(src.width * scale_factor)
            new_height = int(src.height * scale_factor)

            # Redimensionar y guardar el nuevo archivo
            data = src.read(
                out_shape=(
                    src.count,
                    new_height,
                    new_width
                ),
                resampling=Resampling.bilinear
            )

            transform = src.transform * src.transform.scale(
                (src.width / data.shape[-1]),
                (src.height / data.shape[-2])
            )

            # Guardar el archivo reducido
            with rasterio.open(
                output_path,
                'w',
                driver='GTiff',
                height=new_height,
                width=new_width,
                count=src.count,
                dtype=data.dtype,
                crs=src.crs,
                transform=transform,
            ) as dst:
                dst.write(data)

    # Rutas de los archivos GeoTIFF
    input_geotiff_path = 'Pea-Colorada-PeCo_Orto_Feb2024-57-dsm.tif'
    reduced_geotiff_path = 'reduced_Pea-Colorada-PeCo_Orto_Feb2024-57-dsm.tif'

    # Reducir el tamaño del archivo GeoTIFF a un 50% de su tamaño original
    reduce_geotiff_with_rasterio(input_geotiff_path, reduced_geotiff_path, scale_factor=0.5)

    # Cargar el archivo GeoTIFF reducido
    with rasterio.open(reduced_geotiff_path) as src:
        bounds = src.bounds
        img = src.read(1)

    # Verificar las dimensiones y los límites del archivo GeoTIFF
    print(f"GeoTIFF cargado con dimensiones: {img.shape}")
    print(f"Límites del GeoTIFF: {bounds}")

    # Calcular el centro del mapa
    latitude_mean = filtered_data['latitude'].mean()
    longitude_mean = filtered_data['longitude'].mean()

    if pd.isna(latitude_mean) or pd.isna(longitude_mean):
        print("No se pueden calcular las medias de latitud o longitud. Verifica los datos.")
    else:
        # Mapa de velocidad
        speed_map_center = [latitude_mean, longitude_mean]
        speed_map = folium.Map(location=speed_map_center, zoom_start=13)

        # Añadir el GeoTIFF al mapa usando ImageOverlay
        folium.raster_layers.ImageOverlay(
            image=img,
            bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
            opacity=0.6,
            interactive=True,
            cross_origin=False,
            zindex=1
        ).add_to(speed_map)

        folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri',
            name='Esri Satellite',
            overlay=False,
            control=True
        ).add_to(speed_map)

        Fullscreen(position='topright', title='Pantalla completa', title_cancel='Salir de pantalla completa').add_to(speed_map)

        heatmap_data = filtered_data[['latitude', 'longitude', 'Speed']].dropna().values.tolist()
        HeatMap(
            heatmap_data,
            min_opacity=0.2,  # Reducir la opacidad mínima para hacer los puntos menos prominentes
            radius=8,  # Reducir el radio de los puntos
            blur=6,  # Reducir el desenfoque de los puntos
            max_zoom=1,
            gradient={0.0: 'blue', 0.5: 'lime', 1.0: 'red'}
        ).add_to(speed_map)

        FloatImage('colorbar.png', bottom=5, left=5).add_to(speed_map)

        # Mostrar el mapa
        folium_static(speed_map)


Registros cargados: 1000 filas y 7 columnas
Después de convertir UTM a lat/lon:
                 time             j              i    Elevation  Speed  \
0 2020-01-01 22:27:09  2.143059e+06  595533.288961  1108.694285   61.0   
1 2020-01-01 22:27:10  2.143059e+06  595540.878338  1113.924285   22.0   
2 2020-01-01 22:43:12  2.142916e+06  594483.930769   939.544285   14.0   
3 2020-01-01 22:45:50  2.143009e+06  595432.437744  1078.554285   47.0   
4 2020-01-01 22:45:51  2.143006e+06  595424.912236  1081.164285   17.0   

                         full_name               Alarma   latitude   longitude  
0  Leonardo Guadalupe Lecona Pérez  Exceso de Velocidad  19.379396 -104.090268  
1  Leonardo Guadalupe Lecona Pérez  Exceso de Velocidad  19.379394 -104.090195  
2        Emmanuel Lizardi Sandoval        Rt R Susp Cyl  19.378149 -104.100267  
3  Leonardo Guadalupe Lecona Pérez  Exceso de Velocidad  19.378948 -104.091230  
4  Leonardo Guadalupe Lecona Pérez  Exceso de Velocidad  19.378920 -10

2024-08-01 11:41:49.420 
  command:

    streamlit run c:\Users\mswde\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
